##### This is a demo program which uses dltHub within MS Fabric environment to load breaches raw data from https://haveibeenpwned.com/api/v2/breaches and load into MS Fabric lakehouse in CSV

In [ ]:
!pip install dlt

In [ ]:
!pip install "dlt[filesystem]"

In [ ]:
import dlt
import requests

##### dlt resource which fetches breaches data from API and returns data. write_disposition is set to replace and this should be as per use cases. Incremental load makes more sense, however write_disposition is set to replace just for this demo purpose.

In [ ]:
@dlt.resource(write_disposition="replace", table_name="breaches")
def fetch_data():
    api_url = "https://haveibeenpwned.com/api/v2/breaches"
    response = requests.get(api_url)
    yield response.json()

##### Load data to lakehouse. In a medallion architecture, think this is a raw layer

In [ ]:
from dlt.destinations import filesystem
pipeline = dlt.pipeline(
                        pipeline_name="load_raw_breaches",
                        destination=filesystem(
                            bucket_url="/lakehouse/default/Files",
                        )
                       )

load_info = pipeline.run(fetch_data(),
                         table_name="breaches",
                         loader_file_format="csv"
                         )

print(load_info)